# Glove pretrained in the dataset

https://medium.com/@japneet121/word-vectorization-using-glove-76919685ee0b

In [1]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [2]:
from methods.baseline import Baseline

Using TensorFlow backend.


In [3]:
MAX_SEQUENCE_LENGTH_T = 100 # 40
MAX_SEQUENCE_LENGTH_D = 500 # 200
EMBEDDING_DIM = 300
MAX_NB_WORDS = 2000

In [4]:
# Domain to use
DOMAIN = 'eclipse'
# Dataset paths
DIR = 'data/processed/{}'.format(DOMAIN)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Glove embeddings
GLOVE_DIR='data/embed'
# Save model
SAVE_PATH = 'baseline_feature@number_of_epochs@epochs_64batch({})'.format(DOMAIN)
SAVE_PATH_FEATURE = 'baseline_feature_@number_of_epochs@epochs_64batch({})'.format(DOMAIN)

In [5]:
baseline = Baseline(DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D)

In [6]:
baseline.load_ids(DIR)
len(baseline.bug_ids)

Reading bug ids


212512

In [7]:
def data_padding(data, max_seq_length):
    seq_lengths = [len(seq) for seq in data]
    seq_lengths.append(6)
    max_seq_length = min(max(seq_lengths), max_seq_length)
    padded_data = np.zeros(shape=[len(data), max_seq_length])
    for i, seq in enumerate(data):
        seq = seq[:max_seq_length]
        for j, token in enumerate(seq):
            padded_data[i, j] = int(token)
    return padded_data.astype(np.int)

In [8]:
import _pickle as pickle

def load_bugs(baseline):   
    removed = []
    baseline.corpus = []
    baseline.sentence_dict = {}
    baseline.bug_set = {}
    title_padding, desc_padding = [], []
    for bug_id in tqdm(baseline.bug_ids):
        try:
            bug = pickle.load(open(os.path.join(baseline.DIR, 'bugs', '{}.pkl'.format(bug_id)), 'rb'))
            title_padding.append(bug['title_word'])
            desc_padding.append(bug['description_word'])
            baseline.bug_set[bug_id] = bug
            #break
        except:
            removed.append(bug_id)
    
    if len(removed) > 0:
        for x in removed:
            baseline.bug_ids.remove(x)
        baseline.removed = removed
        print("{} were removed. To see the list call self.removed".format(len(removed)))

In [9]:
%%time

load_bugs(baseline)


CPU times: user 6.71 s, sys: 1.56 s, total: 8.27 s
Wall time: 8.38 s


### Read the corpus from bugs

In [10]:
lines_title = [baseline.bug_set[bug_id]['title'] for bug_id in baseline.bug_ids]
lines_desc = [baseline.bug_set[bug_id]['description'] for bug_id in baseline.bug_ids]

lines = []
for title, desc in zip(lines_title, lines_desc):
    lines.append(title)
    lines.append(desc)
    
lines = [line.split(' ') for line in lines]

In [11]:
len(lines)

425024

## Glove embedding

In [12]:
#importing the glove library
from glove import Corpus, Glove
# creating a corpus object
corpus = Corpus() 
#training the corpus to generate the co occurence matrix which is used in GloVe
corpus.fit(lines, window=3)
#creating a Glove object which will use the matrix created in the above lines to create embeddings
#We can set the learning rate as it uses Gradient Descent and number of components
glove = Glove(no_components=300, learning_rate=0.05)
 
glove.fit(corpus.matrix, epochs=100, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
#glove.save('glove.model')

Performing 100 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99


In [13]:
#glove_load = Glove.load('glove.model')

In [14]:
glove.word_vectors[glove.dictionary['eclipse']]

array([-0.48383074,  0.0568191 , -0.36339908,  0.385653  ,  0.51172982,
       -0.11968889,  0.15625221, -0.86259061, -0.27925776, -0.049257  ,
       -0.23049378,  0.0921736 , -0.32677491,  0.31964339, -0.14420828,
        0.01542944,  0.1928976 ,  0.24927734,  0.49672349, -0.25302636,
        0.34276281, -0.66503552, -0.25311815,  0.51157897,  0.26966073,
       -0.43531633,  0.06269993,  0.70071834, -0.31591375, -0.10548172,
       -0.59642175,  0.33029841,  0.14162732,  0.21868475, -0.54065476,
       -0.04273756,  0.18382891,  0.30776916,  0.36066793, -0.20152827,
       -0.12206737,  0.09137368,  0.53385211, -0.2441726 ,  0.15307487,
       -0.01601128, -0.0107764 ,  0.11395006,  0.19539965, -0.02063496,
       -0.77509077,  0.48296848,  0.62096877,  0.01056026, -0.23755392,
       -0.3721398 ,  0.95448282, -0.14374212,  0.66784503, -0.19153906,
       -0.10150599,  0.61680046, -0.17187293, -0.00879854,  0.56220074,
       -1.41199406,  0.6076707 ,  0.36148913,  0.20154918, -0.12

### Save dataset vocabulary embedding

In [15]:
import _pickle as pickle

In [18]:
vocab_embed = {}
for word in glove.dictionary:
    vocab_embed[word] = glove.word_vectors[glove.dictionary[word]].tolist()
    
with open(os.path.join(baseline.DIR, 'vocab_embed.pkl'), 'wb') as f:
      pickle.dump(vocab_embed, f)

In [17]:
vocab_embed['eclipse']

[-0.48383073588655934,
 0.056819095860730646,
 -0.36339908251636505,
 0.3856529974962266,
 0.5117298240737226,
 -0.11968888839281197,
 0.15625220880106871,
 -0.8625906076988131,
 -0.27925776297262905,
 -0.04925700104973989,
 -0.23049377987610523,
 0.09217360126121264,
 -0.32677491009377974,
 0.319643389236405,
 -0.14420828233154537,
 0.015429441736187098,
 0.1928976026450644,
 0.24927733779618697,
 0.49672348764141167,
 -0.2530263597951073,
 0.34276281274370907,
 -0.6650355184474946,
 -0.25311815201090426,
 0.511578970224598,
 0.26966073027097154,
 -0.4353163278813191,
 0.06269992511182829,
 0.7007183381923656,
 -0.31591375471471506,
 -0.10548171784890126,
 -0.5964217519969777,
 0.33029840906359614,
 0.14162732326452235,
 0.21868474765228005,
 -0.5406547602612749,
 -0.042737563384220634,
 0.18382891027074427,
 0.3077691641393479,
 0.3606679297197239,
 -0.20152827253450342,
 -0.12206737392089109,
 0.09137368132536239,
 0.5338521108980067,
 -0.24417260154795542,
 0.15307486516154648,
 -0